# Senior Cubers Worldwide - Weekly Competition

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/scw-comp/

## Initialisation

Basic approach to determine the project directory

In [1]:
import os, sys

projdir = os.path.realpath(os.path.join(sys.path[0], '..'))

## Global Definitions

List of events, descriptions, prize groups, etc

In [2]:
title = 'Senior Cubers Worldwide - Weekly Competition'

eventNames = \
[
    '333',
    '222',
    '444',
    '555',
    '666',
    '777',
    '333oh',
    'minx',
    'pyram',
    'skewb',
    'sq1',
    'clock',
    '333bf',
    '444bf',
    '555bf',
    '333fm'
]

eventTitles = \
{
    '333': ['3x3x3', '3x3x3', '3x3x3'],
    '222': ['2x2x2', '2x2x2', '2x2x2'],
    '333oh': ['3x3x3 One-Handed', '3x3x3 OH', 'OH'],
    'minx':  ['Megaminx', 'Megaminx', 'Mega'],

    '444': ['4x4x4', '4x4x4', '4x4x4'],
    '555': ['5x5x5', '5x5x5', '5x5x5'],
    '666': ['6x6x6', '6x6x6', '6x6x6'],
    '777': ['7x7x7', '7x7x7', '7x7x7'],

    'pyram': ['Pyraminx', 'Pyraminx', 'Pyra'],
    'skewb': ['Skewb', 'Skewb', 'Skewb'],
    'sq1': ['Square-1', 'Square-1', 'Sq-1'],
    'clock': ['Clock', 'Clock', 'Clock'],

    '333bf': ['3x3x3 Blindfolded', '3x3x3 BLD', '3BLD'],
    '444bf': ['4x4x4 Blindfolded', '4x4x4 BLD', '4BLD'],
    '555bf': ['5x5x5 Blindfolded', '5x5x5 BLD', '5BLD'],
    '333fm': ['3x3x3 Fewest Moves', '3x3x3 FMC', 'FMC']
}

prizeGroups = \
[
    ['333', '222', '333oh', 'minx'],
    ['444', '555', '666', '777'],
    ['pyram', 'skewb', 'sq1', 'clock'],
    ['333bf', '444bf', '555bf', '333fm']
]

eventPrizeGroups = {}
for groupNo in range(len(prizeGroups)):
    for event in prizeGroups[groupNo]:
        eventPrizeGroups[event] = groupNo

## Formatting Functions

Functions to convert results to and from seconds or display an age category

In [3]:
from Common_Functions import *

In [4]:
def getCorrectName(name):
    '''Handle people who regularly enter their name differently'''
    
    safeName = getSafeName(name)

    if safeName in safeNames:
        return safeNames[safeName]
    else:
        return name

In [5]:
def writeGoogleSiteTag(f):
    '''Write Google Site Tag'''

    comment = '<!-- Global site tag (gtag.js) - Google Analytics -->'
    src = '<script async src="https://www.googletagmanager.com/gtag/js?id=UA-86348435-3"></script>'
    gtag = "<script>window.dataLayer = window.dataLayer || []; function gtag() {dataLayer.push(arguments);} gtag('js', new Date()); gtag('config', 'UA-86348435-3');</script>"

    f.write('\n{}\n{}\n{}\n'.format(comment, src, gtag))

## Generic Class

Generic class to ensure that all custom classes are printable

In [6]:
class Printable:
    def __repr__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

## Result Class

Simple class for a competition result

In [7]:
class Result(Printable):
    def __init__(self, date, rank, name, age, result, single, average, awards, solves, link):
        self.date, self.rank = date, rank
        self.name, self.age = name, age
        self.result, self.single, self.average = result, single, average
        self.awards = awards
        self.solves, self.link = solves, formatFacebookLink(link)
        
        
    def saveHeader(self, f, eventName):
        '''Save header for table'''
        
        f.write('| # | Name | Age | Single |')
        if eventName.lower() != '333fm':
            f.write(' Average |')
        f.write(' Awards |')
        for i in range(len(self.solves)):
            f.write(' Solve {} |'.format(i + 1))
        if eventName.lower() == '333fm':
            f.write(' Solution |\n')
        else:
            f.write(' Video |\n')

        f.write('| :--: | -- | :--: |')
        if eventName.lower() != '333fm':
            f.write(' --: | --: |')
        else:
            f.write(' :--: |')
        f.write(' :--: |')
        for i in self.solves:
            f.write(' --: |')
        f.write(' :-- |\n')

        
    def saveResult(self, f, eventName):
        '''Save header for table'''
        
        link = '[{}](../../persons/{}.md)'.format(self.name, getSafeName(self.name))
        f.write('| {:d} | {} | {} |'.format(self.rank, link, formatAge(self.age)))
        f.write(' {} |'.format(formatResult(self.single, eventName)))
        if eventName.lower() != '333fm':
            f.write(' {} |'.format(formatResult(self.average, eventName)))
        f.write(' {} |'.format(self.awards))
        for solve in self.solves:
            f.write(' {} |'.format(formatResult(solve, eventName)))
        if 'http' in self.link:
            f.write(' [Link]({}) |'.format(self.link))
        else:
            f.write(' |')
        f.write('\n')

## Event Class

Simple class to hold a list of results

In [8]:
class Event(Printable):
    def __init__(self, name):
        self.name = name
        self.results = []
        self.maxSolves = 0

        self.longName = eventTitles[name][0]
        self.mediumName = eventTitles[name][1]
        self.shortName = eventTitles[name][2]

        
    def addResult(self, result):
        self.results.append(result)

## Person Class

Results for a single person 

In [9]:
import unicodedata

defaultSkipDate = '9999-12-31'

class Person(Printable):
    def __init__(self, id, age, name = None, wcaId = None):
        self.id = id
        self.age = age
        self.name = name
        self.aliases = []
        self.wcaId = wcaId
        self.skipDraw = defaultSkipDate
        self.bestSingles = {}
        self.bestSinglesExtra = {}
        self.bestAverages = {}
        self.bestAveragesExtra = {}
        self.events = {}


    def isBestSingle(self, eventName, result):
        isBest = False
        
        if result.single:
            # Ignoring age
            if eventName in self.bestSingles:
                if self.bestSingles[eventName].single > 0:
                    if result.single > 0 and self.bestSingles[eventName].single >= result.single:
                        self.bestSingles[eventName] = result
                        isBest = True
                else:
                    if result.single > 0 and self.bestSingles[eventName].single <= result.single:
                        self.bestSingles[eventName] = result
                        isBest = True
            else:
                self.bestSingles[eventName] = result
                isBest = True

            # Considering age
            if result.age >= 40:
                startAge = 40
            else:
                startAge = result.age
            for age in range(startAge, 100, 10):
                if age == result.age or (age >= 40 and age <= result.age):
                    if age in self.bestSinglesExtra:
                        if eventName in self.bestSinglesExtra[age]:
                            if self.bestSinglesExtra[age][eventName].single > 0:
                                if result.single > 0 and self.bestSinglesExtra[age][eventName].single >= result.single:
                                    self.bestSinglesExtra[age][eventName] = result
                                    isBest = True
                            else:
                                if result.single > 0 and self.bestSinglesExtra[age][eventName].single <= result.single:
                                    self.bestSinglesExtra[age][eventName] = result
                                    isBest = True
                        else:
                            self.bestSinglesExtra[age][eventName] = result
                            isBest = True
                    else:
                            self.bestSinglesExtra[age] = {}
                            self.bestSinglesExtra[age][eventName] = result
                            isBest = True

        return isBest
    
    
    def isBestAverage(self, eventName, result):
        isBest = False
     
        if result.average:
            # Ignoring age
            if eventName in self.bestAverages:
                if self.bestAverages[eventName].average > 0:
                    if result.average > 0 and self.bestAverages[eventName].average >= result.average:
                        self.bestAverages[eventName] = result
                        isBest = True
                else:
                    if result.average > 0 and self.bestAverages[eventName].average <= result.average:
                        self.bestAverages[eventName] = result
                        isBest = True
            else:
                self.bestAverages[eventName] = result
                isBest = True

            # Considering age
            if result.age >= 40:
                startAge = 40
            else:
                startAge = result.age
            for age in range(startAge, 100, 10):
                if age == result.age or (age >= 40 and age <= result.age):
                    if age in self.bestAveragesExtra:
                        if eventName in self.bestAveragesExtra[age]:
                            if self.bestAveragesExtra[age][eventName].average > 0:
                                if result.average > 0 and self.bestAveragesExtra[age][eventName].average >= result.average:
                                    self.bestAveragesExtra[age][eventName] = result
                                    isBest = True
                            else:
                                if result.average > 0 and self.bestAveragesExtra[age][eventName].average <= result.average:
                                    self.bestAveragesExtra[age][eventName] = result
                                    isBest = True
                        else:
                            self.bestAveragesExtra[age][eventName] = result
                            isBest = True
                    else:
                            self.bestAveragesExtra[age] = {}
                            self.bestAveragesExtra[age][eventName] = result
                            isBest = True

        return isBest

    
    def addResult(self, eventName, result):
        if eventName in self.events:
            event = self.events[eventName]
        else:
            event = Event(eventName)
            self.events[eventName] = event

        event.addResult(result)

        if result.age > self.age:
            self.age = result.age
            
            
    def saveEventHeader(self, f, event):
        '''Save header for table'''
        
        f.write('| Date | Age | Single |')
        if event.name.lower() != '333fm':
            f.write(' Average |')
        f.write(' Awards |')
        for i in range(event.maxSolves):
            f.write(' Solve {} |'.format(i + 1))
        if event.name.lower() == '333fm':
            f.write(' Solution |\n')
        else:
            f.write(' Video |\n')

        f.write('| :--: | :--: |')
        if event.name.lower() != '333fm':
            f.write(' --: | --: |')
        else:
            f.write(' :--: |')
        f.write(' :--: |')
        for i in range(event.maxSolves):
            f.write(' --: |')
        f.write(' :-- |\n')

        
    def saveEventResult(self, f, event, result, count):
        '''Save header for table'''
        
        link = '[{}](../../results/{}/{}.md)'.format(result.date, getSafeName(event.name), result.date)
        f.write('| {} | {} |'.format(link, formatAge(result.age)))
        if '⚡' in result.awards or count == 1:
            highlight = '**'
        else:
            highlight = ''
        f.write(' {} |'.format(formatResult(result.single, event.name, highlight)))
        if event.name.lower() != '333fm':
            if '🔥' in result.awards or count == 1:
                highlight = '**'
            else:
                highlight = ''
            f.write(' {} |'.format(formatResult(result.average, event.name, highlight)))
        f.write(' {} |'.format(result.awards))
        for i in range(event.maxSolves):
            if len(result.solves) > i:
                if result.solves[i] > 0 and result.solves[i] == result.single and ('⚡' in result.awards or count == 1):
                    highlight = '**'
                else:
                    highlight = ''
                f.write(' {} |'.format(formatResult(result.solves[i], event.name, highlight)))
            else:
                f.write(' - |')
        if 'http' in result.link:
            f.write(' [Link]({}) |'.format(result.link))
        else:
            f.write(' |')
        f.write('\n')

        
    def saveProfile(self):
        '''Save profile as markdown'''

        safeName = getSafeName(self.name)
        
        docsDir = os.path.join(projdir, 'docs', 'persons')
        if not os.path.exists(docsDir):
            os.makedirs(docsDir)

        outFile = os.path.join(docsDir, safeName + '.md')

        with open(outFile, 'w') as f:
            
            f.write('## {}\n'.format(title))
            f.write('### {}'.format(self.name))
            if self.wcaId:
                f.write(' - [{}](https://www.worldcubeassociation.org/persons/{})'.format(self.wcaId, self.wcaId))
            f.write('\n\n'.format(self.name))

            f.write('| Event | Single | Average |\n')
            f.write('| -- | --: | --: |\n')
            
            for eventName in eventNames:
                if eventName in self.events:
                    event = Event(eventName)
                    f.write('| [{}]({}/{}.md) |'.format(event.mediumName, safeName, eventName))
                    if eventName in self.bestSingles:
                        f.write(' {} |'.format(formatResult(self.bestSingles[eventName].single, eventName)))
                    else:
                        f.write(' - |')
                    if eventName in self.bestAverages:
                        f.write(' {} |'.format(formatResult(self.bestAverages[eventName].average, eventName)))
                    else:
                        f.write(' - |')
                    f.write('\n')

            writeGoogleSiteTag(f)

        for eventName in eventNames:
            if eventName in self.events:
                docsDir = os.path.join(projdir, 'docs', 'persons', safeName)
                if not os.path.exists(docsDir):
                    os.makedirs(docsDir)

                outFile = os.path.join(docsDir, eventName + '.md')

                with open(outFile, 'w') as f:
                    event = self.events[eventName]

                    f.write('## {}\n'.format(title))
                    f.write('### {} - {}'.format(self.name, event.longName))
                    if self.wcaId:
                        f.write(' - [{}](https://www.worldcubeassociation.org/persons/{}?event={})'.format(
                            self.wcaId, self.wcaId, eventName))
                    f.write('\n\n'.format(self.name))

                    event.maxSolves = 0
                    for result in event.results:
                        if len(result.solves) > event.maxSolves:
                            event.maxSolves = len(result.solves)

                    self.saveEventHeader(f, event)
                    count = len(event.results)
                    for result in reversed(event.results):
                        self.saveEventResult(f, event, result, count)
                        count -= 1

                    f.write('\n')

                    writeGoogleSiteTag(f)

## Competition Class

Results for a single competition 

In [10]:
from datetime import datetime

import os
import glob
import csv

import Levenshtein
import random

class Competition(Printable):
    def __init__(self, dataDir):
        self.dataDir = dataDir
        self.date = os.path.basename(dataDir)
        self.events = {}

        
    def saveEvent(self, eventName):
        '''Save results as markdown'''

        event = self.events[eventName]
        results = event.results
        
        docsDir = os.path.join(projdir, 'docs', 'results', getSafeName(eventName))
        if not os.path.exists(docsDir):
            os.makedirs(docsDir)

        outFile = os.path.join(docsDir, self.date + '.md')

        with open(outFile, 'w') as f:
            
            f.write('## {}\n'.format(title))
            f.write('### {} {}\n\n'.format(event.longName, self.date))
            
            results[0].saveHeader(f, eventName)
            for result in results:
                result.saveResult(f, eventName)

            writeGoogleSiteTag(f)


    def saveEvents(self):
        '''Save results as markdown'''

        for eventName in self.events:
            self.saveEvent(eventName)


    def drawPrizes(self):
        '''Draw prizes from hat - quick hack for the first week'''

        prizeDraw = []
        for group in range(4):
            prizeDraw += self.prizeDraw[group]
        prizeDraw = sorted(prizeDraw)

        winners = []
        random.seed(a = self.date)
        while len(winners) < 4:
            winner = random.choice(prizeDraw)
            winners.append(winner)
            while winner in prizeDraw:
                prizeDraw.remove(winner)

        print("Prize winners {}:".format(self.date))
        names = []
        for winner in winners:
            for person in persons:
                if persons[person].id == winner:
                    names.append(person)
        print("- {}".format(", ".join(names)))


    def processSheets(self, latest):
        '''Process a spreadsheet which has been downloaded from Google Sheets'''

        csvFiles = glob.glob(os.path.join(self.dataDir, '*.csv'))

        self.prizeDraw = [[], [], [], []]
        
        for fn in csvFiles:
            with open(fn, 'r') as f:
                csvReader = csv.reader(f)

                # Variable columns
                indexName = None
                indexWcaId = None
                indexAge = None
                indexBest = None
                indexAverage = None
                indexSolves = []
                indexLink = None

                # Counters, etc
                rowNo = 1
                prevResult = prevSingle = 0
                prevSeniorResult = prevSeniorSingle = 0
                rank = seniorRank = 0
                nextRank = nextSeniorRank = 1
                results = []

                eventName = os.path.basename(fn).split('.')[0]
                event = Event(eventName)
                self.events[eventName] = event

                for values in csvReader:            
                    # Process header row
                    if rowNo == 1:
                        for i in range(len(values)):
                            if values[i] == 'Name':
                                indexName = i
                            elif values[i] == 'WCA ID':
                                indexWcaId = i
                            elif values[i] == 'Age':
                                indexAge = i
                            elif values[i] == 'Best':
                                indexBest = i
                            elif values[i] == 'Average':
                                indexAverage = i
                            elif values[i] == 'Result':
                                indexBest = i
                            elif str(values[i]).startswith('Solve'):
                                indexSolves.append(i)
                            elif 'link' in str(values[i]).lower():
                                indexLink = i
        
                    # Process result row - must be after header on row 3
                    else:
                        # Fixed fields
                        name = getCorrectName(values[indexName].strip())
                        if indexWcaId:
                            wcaId = values[indexWcaId].strip()
                            if wcaId == "":
                                wcaId = None
                        else:
                            wcaId = None
                        
                        try:
                            age = values[indexAge].lower()
                            if 'under' in age:
                                age = int(age.replace('under', '').replace(' ', '')) - 10
                            elif 'over' in age:
                                age = int(age.replace('over', '').replace(' ', ''))
                            elif '<' in age:
                                age = int(age.replace('<', '').replace(' ', '')) - 10
                            elif '>' in age:
                                age = int(age.replace('>', '').replace(' ', ''))
                            else:
                                age = int(age.replace('+', ''))
                        except:
                            print('ERROR: Issue with age for {} in {} {}'.format(name, eventName, self.date))

                        # Retrieve single
                        if indexBest:
                            try:
                                single = numSeconds(values[indexBest])
                            except:
                                print('ERROR: Issue with best for {} in {} {}'.format(name, eventName, self.date))
                        else:
                            single = None

                        # Retrieve average
                        if indexAverage:
                            try:
                                average = numSeconds(values[indexAverage])
                            except:
                                print('ERROR: Issue with average for {} in {} {}'.format(name, eventName, self.date))
                        else:
                            average = None

                        # Process solves
                        solves = []
                        for indexSolve in indexSolves:
                            try:
                                value = numSeconds(values[indexSolve])
                            except:
                                print('ERROR: Issue with result for {} in {} {}'.format(name, eventName, self.date))
                                raise
                                
                            # Some old CSV files do not contain the single
                            if value and value > 0:
                                if single is None or single > value:
                                    single = value

                            solves.append(value)

                        # Awards will be determined later
                        awards = ''
                        link = values[indexLink].strip()

                        # Determine result in seconds and rank
                        if eventName == '333fm' or eventName.endswith('bf'):
                            thisResult = single
                        else:
                            thisResult = average

                        if thisResult != prevResult or single != prevSingle:
                            rank = nextRank
                        nextRank += 1

                        if age >= 40:
                            if thisResult != prevSeniorResult or single != prevSeniorSingle:
                                seniorRank = nextSeniorRank
                            nextSeniorRank += 1

                        # Check person
                        if name in persons:
                            person = persons[name]
                        else:
                            safeName = getSafeName(name)
                            if safeName in safeNames:
                                name = safeNames[safeName]
                                person = persons[name]
                            else:
                                for person in persons:
                                    if (Levenshtein.distance(person, name) < 5):
                                        personBytes = len(person)
                                        nameBytes = len(name)
                                        print("WARNING: Similar names - '{}' ({}) in {} vs existing '{}' ({})".format(
                                            name, nameBytes, eventName, person, personBytes))
                                id = len(persons) + 1
                                person = Person(id, age, name, wcaId)
                                persons[name] = person

                        # Add name to prize draw
                        prizeGroup = eventPrizeGroups[eventName]
                        if person.skipDraw > self.date and self.prizeDraw[prizeGroup].count(person.id) < 2:
                            self.prizeDraw[prizeGroup].append(person.id)
                        
                        # Report age change
                        if age > person.age and latest:
                            print('Info: Higher age for {} in {} {}'.format(name, eventName, self.date))
                        elif age < person.age and latest:
                            print('ERROR: Lower age for {} in {} {}'.format(name, eventName, self.date))

                        # Update WCA ID
                        if person.wcaId is None and wcaId is not None:
                            person.wcaId = wcaId
                        
                        # Record result
                        result = Result(self.date, rank, name, age, thisResult, single, average, awards, solves, link)
                        results.append(result)
                        person.addResult(eventName, result)
                        event.addResult(result)

                        # Check rank
                        if thisResult is None:
                            print('ERROR: No result for {} in {} {}'.format(name, eventName, self.date))
                        if (thisResult > 0 and thisResult < prevResult):
                            print('ERROR: Order is incorrect for {} in {} {}'.format(name, eventName, self.date))

                        # Determine rewards
                        awards = []
                        if rank == 1 and single > 0:
                            awards.append('🏆')
                        if age >= 40 and single > 0:
                            if seniorRank == 1:
                                awards.append('🥇')
                            elif seniorRank == 2:
                                awards.append('🥈')
                            elif seniorRank == 3:
                                awards.append('🥉')
                        if person.isBestAverage(eventName, result):
                            awards.append('🔥')
                        if person.isBestSingle(eventName, result):
                            awards.append('⚡')
                        result.awards = ' '.join(awards)

                        # Copy list of solves prior to manipulation
                        tmpSolves = solves.copy()

                        # Remove DNS - e.g. doing Mo3 rather than Ao5
                        while -2 in tmpSolves:
                            tmpSolves.remove(-2)

                        # Remove n/a - e.g. doing Mo3 rather than Ao5
                        while 0 in tmpSolves:
                            tmpSolves.remove(0)

                        # Sort remaining solves
                        for i in range(len(tmpSolves)):
                            if tmpSolves[i] < 0:
                                tmpSolves[i] = 999999
                        tmpSolves = sorted(tmpSolves)

                        # Check Ao5
                        if (len(tmpSolves)) == 5:
                            if tmpSolves[3] == 999999:
                                if average != -1:
                                    print('WARNING: DNF average for {} in {} {}'.format(name, eventName, self.date))
                            else:
                                diff = round(average - (tmpSolves[1] + tmpSolves[2] + tmpSolves[3]) / 3, 2)
                                if diff < -0.01 or diff > 0.01:
                                    print('WARNING: Incorrect Ao5 for {} in {} {} - expected {} {}'.format(name, eventName, self.date, average - diff, tmpSolves))

                        # Check Mo3
                        if (len(tmpSolves)) == 3:
                            if tmpSolves[2] == 999999:
                                if average != -1:
                                    print('WARNING: DNF mean for {} in {} {}'.format(name, eventName, self.date))
                            else:
                                diff = round(average - (tmpSolves[0] + tmpSolves[1] + tmpSolves[2]) / 3, 2)
                                if diff < -0.01 or diff > 0.01:
                                    print('WARNING: Incorrect Mo3 for {} in {} {} - expected {} {}'.format(name, eventName, self.date, average - diff, tmpSolves))

                        prevResult = thisResult
                        prevSingle = single

                        if age >= 40:
                            prevSeniorResult = thisResult
                            prevSeniorSingle = single

                    rowNo += 1

## Functions to Save Results

Generate markdown files for results, etc.

In [11]:
def saveProfiles():
    # Remove existing profile pages
    for i in glob.glob(os.path.join(projdir, 'docs', 'persons', '*.md')):
        os.unlink(i)

    # Save profile pages
    for person in sorted(persons):
        persons[person].saveProfile()

In [12]:
def saveCompetitors():
    for eventName in eventNames:
        docsDir = os.path.join(projdir, 'docs', 'results', getSafeName(eventName))
        if not os.path.exists(docsDir):
            os.makedirs(docsDir)

        outFile = os.path.join(docsDir, 'persons.md')

        event = Event(eventName)

        with open(outFile, 'w') as f:

            f.write('## {}\n'.format(title))
            f.write('### {} Competitors\n\n'.format(event.longName))

            for age in range(100, 0, -10):
                found = False

                for personName in sorted(persons):
                    person = persons[personName]
                    if age in person.bestSinglesExtra and eventName in person.bestSinglesExtra[age]:
                        if found == False:
                            f.write('#### {}\n\n'.format(formatAgeLong(age)))

                            f.write('| Name | Age | Single |')
                            if eventName.lower() != '333fm':
                                f.write(' Average |')
                            f.write('\n')

                            f.write('| -- |')
                            if eventName.lower() != '333fm':
                                f.write(' :--: | --: | --: |')
                            else:
                                f.write(' :--: | :--: |')
                            f.write('\n')

                            found = True

                        link = '[{}](../../persons/{}.md)'.format(person.name, getSafeName(person.name))
                        f.write('| {} |'.format(link))
                        for tmpAge in person.bestSinglesExtra.keys():
                            if eventName in person.bestSinglesExtra[tmpAge]:
                                maxAge = tmpAge
                        f.write(' {} | {} |'.format(formatAge(maxAge), formatResult(person.bestSinglesExtra[age][eventName].single, eventName)))
                        if eventName.lower() != '333fm':
                            if eventName in person.bestAverages:
                                f.write(' {} |'.format(formatResult(person.bestAveragesExtra[age][eventName].average, eventName)))
                            else:
                                f.write(' DNF |')
                        f.write('\n')

                if found:
                    f.write('\n')

            writeGoogleSiteTag(f)

In [13]:
def saveBestSingles():
    for eventName in eventNames:
        docsDir = os.path.join(projdir, 'docs', 'results', getSafeName(eventName))
        if not os.path.exists(docsDir):
            os.makedirs(docsDir)

        outFile = os.path.join(docsDir, 'singles.md')

        event = Event(eventName)

        with open(outFile, 'w') as f:

            f.write('## {}\n'.format(title))
            f.write('### {} Singles\n\n'.format(event.longName))

            for age in range(100, 0, -10):
                results = []
                for personName in sorted(persons):
                    person = persons[personName]
                    if age in person.bestSinglesExtra and eventName in person.bestSinglesExtra[age]:
                        if person.bestSinglesExtra[age][eventName].single and person.bestSinglesExtra[age][eventName].single > 0:
                            results.append(person.bestSinglesExtra[age][eventName])

                if results:
                    f.write('#### {}\n\n'.format(formatAgeLong(age)))
                    f.write('| # | Name | Date | Age | Single | Awards |')
                    if eventName.lower() == '333fm':
                        f.write(' Solution |\n')
                    else:
                        f.write(' Video |\n')

                    f.write('| :--: | -- | :--: | :--: |')
                    if eventName.lower() != '333fm':
                        f.write(' --: |')
                    else:
                        f.write(' :--: |')
                    f.write(' :--: | -- |\n')

                    rank = 1
                    prevSingle = 0
                    for result in sorted(results, key=lambda result: result.single):
                        if result.single > prevSingle:
                            f.write('| {} |'.format(rank))
                        else:
                            f.write('| |')
                        link = '[{}](../../persons/{}.md)'.format(result.name, getSafeName(result.name))
                        f.write(' {} |'.format(link))
                        f.write(' [{}]({}.md) | {} |'.format(result.date, result.date, formatAge(result.age)))
                        f.write(' {} | {} |'.format(formatResult(result.single, eventName), result.awards))

                        if 'http' in result.link:
                            f.write(' [Link]({}) |\n'.format(result.link))
                        else:
                            f.write(' |\n')

                        prevSingle = result.single
                        rank += 1

                    f.write('\n')

            writeGoogleSiteTag(f)

In [14]:
def saveBestAverages():
    for eventName in eventNames:
        count = 0

        docsDir = os.path.join(projdir, 'docs', 'results', getSafeName(eventName))
        if not os.path.exists(docsDir):
            os.makedirs(docsDir)

        outFile = os.path.join(docsDir, 'averages.md')

        event = Event(eventName)

        with open(outFile, 'w') as f:

            f.write('## {}\n'.format(title))
            f.write('### {} Averages\n\n'.format(event.longName))

            for age in range(100, 0, -10):
                results = []
                for personName in sorted(persons):
                    person = persons[personName]
                    if age in person.bestAveragesExtra and eventName in person.bestAveragesExtra[age]:
                        if person.bestAveragesExtra[age][eventName].average and person.bestAveragesExtra[age][eventName].average > 0:
                            results.append(person.bestAveragesExtra[age][eventName])

                if results:
                    f.write('#### {}\n\n'.format(formatAgeLong(age)))
                    f.write('| # | Name | Date | Age | Average | Awards |')
                    if eventName.lower() == '333fm':
                        f.write(' Solution |\n')
                    else:
                        f.write(' Video |\n')

                    f.write('| :--: | -- | :--: | :--: |')
                    if eventName.lower() != '333fm':
                        f.write(' --: |')
                    else:
                        f.write(' :--: |')
                    f.write(' :--: | -- |\n')

                    rank = 1
                    prevAverage = 0
                    for result in sorted(results, key=lambda result: result.average):
                        if result.average > prevAverage:
                            f.write('| {} |'.format(rank))
                        else:
                            f.write('| |')
                        link = '[{}](../../persons/{}.md)'.format(result.name, getSafeName(result.name))
                        f.write(' {} |'.format(link))
                        f.write(' [{}]({}.md) | {} |'.format(result.date, result.date, formatAge(result.age)))
                        f.write(' {} | {} |'.format(formatResult(result.average, eventName), result.awards))

                        if 'http' in result.link:
                            f.write(' [Link]({}) |\n'.format(result.link))
                        else:
                            f.write(' |\n')

                        prevAverage = result.average
                        rank += 1
                        count += 1

                    f.write('\n')

            if count == 0:
                f.write('Nobody has an average yet!\n')

            writeGoogleSiteTag(f)

In [15]:
def saveSeniorPodiums():
    for eventName in eventNames:
        docsDir = os.path.join(projdir, 'docs', 'results', getSafeName(eventName))
        if not os.path.exists(docsDir):
            os.makedirs(docsDir)

        outFile = os.path.join(docsDir, 'podiums.md')

        event = Event(eventName)

        with open(outFile, 'w') as f:

            f.write('## {}\n'.format(title))
            f.write('### {} Senior Podiums\n'.format(event.longName))

            for competition in sorted(competitions, reverse = True):
                competition = competitions[competition]

                header = False

                if eventName in competition.events:
                    results = competition.events[eventName].results

                    for result in results:
                        pos = 0
                        if '🥇' in result.awards:
                            pos = 1
                        if '🥈' in result.awards:
                            pos = 2
                        if '🥉' in result.awards:
                            pos = 3
                        if pos > 0:
                            if not header:
                                f.write('#### [{}]({}.md)\n\n'.format(competition.date, competition.date))

                                f.write('| # | Name | Age | Single |')
                                if eventName.lower() == '333fm':
                                    f.write(' Solution |\n')
                                else:
                                    f.write(' Average | Awards | Video |\n')

                                f.write('| :--: | -- | :--: |')
                                if eventName.lower() != '333fm':
                                    f.write(' --: | --: |')
                                else:
                                    f.write(' :--: |')
                                f.write(' :--: | -- |\n')

                                header = True

                            link = '[{}](../../persons/{}.md)'.format(result.name, getSafeName(result.name))
                            f.write('| {} | {} | {} |'.format(pos, link, formatAge(result.age)))
                            f.write(' {} |'.format(formatResult(result.single, eventName)))
                            if eventName.lower() != '333fm':
                                f.write(' {} | {} |'.format(formatResult(result.average, eventName), result.awards))

                            if 'http' in result.link:
                                f.write(' [Link]({}) |\n'.format(result.link))
                            else:
                                f.write(' |\n')

                    f.write('\n')

            writeGoogleSiteTag(f)

In [16]:
def saveIndexPage():
    outFile = os.path.join(projdir, 'docs', 'results', 'README.md')
    with open(outFile, 'w') as f:
        f.write('## {}\n'.format(title))
        f.write('### Historical Archive\n')

        f.write('#### Individuals\n')

        eventList = []
        for eventName in eventNames:
            event = Event(eventName)
            eventList.append('[{}]({}/persons.md)'.format(event.shortName, getSafeName(eventName)))
        f.write('Competitors - {}\n\n'.format(', '.join(eventList)))

        eventList = []
        for eventName in eventNames:
            event = Event(eventName)
            eventList.append('[{}]({}/singles.md)'.format(event.shortName, getSafeName(eventName)))
        f.write('Best Singles - {}\n\n'.format(', '.join(eventList)))

        eventList = []
        for eventName in eventNames:
            event = Event(eventName)
            if eventName.lower() != '333fm':
                eventList.append('[{}]({}/averages.md)'.format(event.shortName, getSafeName(eventName)))
        f.write('Best Averages - {}\n\n'.format(', '.join(eventList)))

        f.write('#### Podiums\n')

        eventList = []
        for eventName in eventNames:
            event = Event(eventName)
            eventList.append('[{}]({}/podiums.md)'.format(event.shortName, getSafeName(eventName)))
        f.write('Seniors - {}\n\n'.format(', '.join(eventList)))

        f.write('#### Results\n')

        for competition in sorted(competitions, reverse = True):
            competition = competitions[competition]
            eventList = []
            for eventName in eventNames:
                if eventName in competition.events:
                    event = Event(eventName)
                    eventList.append('[{}]({}/{}.md)'.format(event.shortName, getSafeName(eventName), competition.date))
            f.write('{} - {}\n\n'.format(competition.date, ', '.join(eventList)))

        f.write('#### Project Code\n')

        url = 'https://github.com/Logiqx/scw-comp'
        f.write('Python code for these reports can be found at [{}]({})\n'.format(url, url))

        writeGoogleSiteTag(f)

## Save Persons

Save updated persons in JSON format

In [17]:
def savePersons():
    # Remove redundant information from persons
    for person in persons:
        if isinstance(persons[person].age, int):
            persons[person].age = formatAge(persons[person].age)

        try:
            if len(persons[person].aliases) == 0:
                del(persons[person].aliases)
        except AttributeError:
            pass

        try:
            if persons[person].skipDraw == defaultSkipDate:
                del(persons[person].skipDraw)
        except AttributeError:
            pass

        try:
            if persons[person].wcaId is None:
                del(persons[person].wcaId)
        except AttributeError:
            pass

        try:
            del(persons[person].name)
        except AttributeError:
            pass

        try:
            del(persons[person].events)
        except AttributeError:
            pass

        try:
            del(persons[person].bestSingles)
            del(persons[person].bestAverages)
        except AttributeError:
            pass

        try:
            del(persons[person].bestSinglesExtra)
            del(persons[person].bestAveragesExtra)
        except AttributeError:
            pass

    # Save persons
    outFile = os.path.join(projdir, 'data', 'persons.json')
    with open(outFile, 'w') as f:
        jsonDump = json.dumps(persons, default=lambda o: o.__dict__, ensure_ascii=False, indent=2, sort_keys=True)
        f.write(jsonDump)

## Main Code

Process all competitions

In [18]:
import glob
import time
import json

pc1 = time.perf_counter()

In [19]:
# Read existing persons from JSON

def as_person(d):
    if 'id' in d:
        person = Person(d['id'], interpretAge(d['age']))
        if 'wcaId' in d:
            person.wcaId = d['wcaId']
        if 'aliases' in d:
            person.aliases = d['aliases']
        if 'skipDraw' in d:
            person.skipDraw = d['skipDraw']
        return person
    else:
        return d

fn = os.path.join(projdir, 'data', 'persons.json')
if os.path.exists(fn):
    with open(fn) as f:
        jsonTxt = f.read()
        persons = json.loads(jsonTxt, object_hook = as_person)

        for person in persons:
            persons[person].name = person
else:
    persons = {}
    
safeNames = {}
for person in persons:
    safeName = getSafeName(person)
    safeNames[safeName] = person

    for alias in persons[person].aliases:
        safeName = getSafeName(alias)
        safeNames[safeName] = person

In [20]:
# Process competition data for all weeks

datePattern = '[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]'

competitions = {}

dataDirs = glob.glob(os.path.join(projdir, 'data', datePattern))
latest = False

for dataDir in dataDirs:
    if dataDir == dataDirs[-1]:
        latest = True

    competition = Competition(dataDir)
    competition.processSheets(latest)
    competition.saveEvents()
    competitions[competition.date] = competition

In [21]:
# Save results
saveCompetitors()
saveBestSingles()
saveBestAverages()
saveSeniorPodiums()
saveIndexPage()

# Save persons
saveProfiles()

# Save data
savePersons()

# Draw prizes
latestComp = sorted(competitions.keys())[-1]
competition = competitions[latestComp]
competition.drawPrizes()

Prize winners 2020-06-23:
- Peter Osudar, Jae Park, Thierry Boisivon, Andy Nicholls


In [22]:
pc2 = time.perf_counter()
print("Reports completed in %0.2f seconds" % (pc2 - pc1))

Reports completed in 6.13 seconds


## All Done!